In [4]:
!python -V

Python 3.9.12


In [5]:
import pandas as pd

In [6]:
import pickle

In [7]:
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [9]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2025/07/15 17:57:00 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/07/15 17:57:00 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='/workspaces/mlops-zoomcamp/03-training/experiment_tracking/mlruns/1', creation_time=1751298403537, experiment_id='1', last_update_time=1751298403537, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [10]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [11]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-02.parquet')

In [12]:
len(df_train), len(df_val)

(46307, 44218)

In [13]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [14]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [15]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [16]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

6.098889826481847

In [17]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [18]:
with mlflow.start_run():

    mlflow.set_tag("developer", "lourdes")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [ ]:
scikitlearn.__version__

In [19]:
import xgboost as xgb

In [20]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [21]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [22]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [23]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/workspaces/mlops-zoomcamp/venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [17:57:04] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:5.98440                           
[1]	validation-rmse:5.63226                           
[2]	validation-rmse:5.56094                           
[3]	validation-rmse:5.52625                           
[4]	validation-rmse:5.50481                           
[5]	validation-rmse:5.49221                           
[6]	validation-rmse:5.48232                           
[7]	validation-rmse:5.48046                           
[8]	validation-rmse:5.47631                           
[9]	validation-rmse:5.47440                           
[10]	validation-rmse:5.46745                          
[11]	validation-rmse:5.46764                          
[12]	validation-rmse:5.46457                          
[13]	validation-rmse:5.46253                          
[14]	validation-rmse:5.46011                          
[15]	validation-rmse:5.45863                          
[16]	validation-rmse:5.45664                          
[17]	validation-rmse:5.45624                          
[18]	valid

/workspaces/mlops-zoomcamp/venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [17:57:32] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.61869                                                    
[1]	validation-rmse:6.72286                                                    
[2]	validation-rmse:6.19897                                                    
[3]	validation-rmse:5.88756                                                    
[4]	validation-rmse:5.72205                                                    
[5]	validation-rmse:5.61272                                                    
[6]	validation-rmse:5.55663                                                    
[7]	validation-rmse:5.52077                                                    
[8]	validation-rmse:5.48971                                                    
[9]	validation-rmse:5.47731                                                    
[10]	validation-rmse:5.46744                                                   
[11]	validation-rmse:5.45982                                                   
[12]	validation-rmse:5.45725            

...

[0]	validation-rmse:5.88584                                                     
[1]	validation-rmse:5.46444                                                     
[2]	validation-rmse:5.40595                                                     
[3]	validation-rmse:5.39704                                                     
[4]	validation-rmse:5.39221                                                     
[5]	validation-rmse:5.38752                                                     
[6]	validation-rmse:5.38427                                                     
[7]	validation-rmse:5.38093                                                     
[8]	validation-rmse:5.37929                                                     
[9]	validation-rmse:5.37910                                                     
[10]	validation-rmse:5.36354                                                    
[11]	validation-rmse:5.36469                                                    
[12]	validation-rmse:5.36339

/workspaces/mlops-zoomcamp/venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [18:19:58] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.81119                                                     
[1]	validation-rmse:6.96461                                                     
[2]	validation-rmse:6.42812                                                     
[3]	validation-rmse:6.09834                                                     
[4]	validation-rmse:5.89634                                                     
[5]	validation-rmse:5.77250                                                     
[6]	validation-rmse:5.69127                                                     
[7]	validation-rmse:5.63926                                                     
[8]	validation-rmse:5.60386                                                     
[9]	validation-rmse:5.57851                                                     
[10]	validation-rmse:5.56204                                                    
[11]	validation-rmse:5.54729                                                    
[12]	validation-rmse:5.53607

/workspaces/mlops-zoomcamp/venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [18:20:30] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[1]	validation-rmse:5.70821                                                     
[2]	validation-rmse:5.53351                                                     
[3]	validation-rmse:5.48136                                                     
[4]	validation-rmse:5.45460                                                     
[5]	validation-rmse:5.44912                                                     
[6]	validation-rmse:5.44235                                                     
[7]	validation-rmse:5.43511                                                     
[8]	validation-rmse:5.42119                                                     
[9]	validation-rmse:5.41452                                                     
[10]	validation-rmse:5.41060                                                    
[11]	validation-rmse:5.41050                                                    
[12]	validation-rmse:5.40830                                                    
[13]	validation-rmse:5.40398

In [24]:
mlflow.xgboost.autolog(disable=True)

In [ ]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/workspaces/mlops-zoomcamp/venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [18:20:38] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:8.56745
[1]	validation-rmse:8.10033
[2]	validation-rmse:7.69739
[3]	validation-rmse:7.35105
[4]	validation-rmse:7.05239
[5]	validation-rmse:6.80020
[6]	validation-rmse:6.58162
[7]	validation-rmse:6.39726
[8]	validation-rmse:6.24197
[9]	validation-rmse:6.11059
[10]	validation-rmse:6.00193
[11]	validation-rmse:5.90700
[12]	validation-rmse:5.82752
[13]	validation-rmse:5.76123
[14]	validation-rmse:5.70306
[15]	validation-rmse:5.65560
[16]	validation-rmse:5.61619
[17]	validation-rmse:5.58065
[18]	validation-rmse:5.55157
[19]	validation-rmse:5.52734
[20]	validation-rmse:5.50858
[21]	validation-rmse:5.48872
[22]	validation-rmse:5.47379
[23]	validation-rmse:5.45957
[24]	validation-rmse:5.44705
[25]	validation-rmse:5.43706
[26]	validation-rmse:5.42713
[27]	validation-rmse:5.41891
[28]	validation-rmse:5.41283
[29]	validation-rmse:5.40819
[30]	validation-rmse:5.40244
[31]	validation-rmse:5.39854
[32]	validation-rmse:5.39446
[33]	validation-rmse:5.39004
[34]	validation-rmse:5.3

2025/07/15 18:21:18 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/workspaces/mlops-zoomcamp/venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [18:21:18] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/07/15 18:21:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


: 

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        